In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Load the LRI AI Spreadsheet data
df = pd.read_csv('data/LRI_AI_Spreadsheet_WVI_NG_9_2025_Deidentified.csv')

print("✅ Data loaded successfully!")
print(f"📊 Dataset shape: {df.shape}")
print(f"🔢 Columns: {len(df.columns)}")
print(f"📝 Rows: {len(df)}")

# Display basic info about the dataset
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)

# Show first few rows
print("\n📋 First 5 rows:")
print(df.head())

print("\n📊 Column names:")
print(df.columns.tolist())

print("\n🔍 Data types:")
print(df.dtypes)


✅ Data loaded successfully!
📊 Dataset shape: (847, 22)
🔢 Columns: 22
📝 Rows: 847

DATASET OVERVIEW

📋 First 5 rows:
   Age  Laterality   Procedure (M) Sphere  (M) Cylinder  (M) Axis  \
0   68            1          1      -0.25         -1.25       109   
1   68            2          1      -1.75         -0.75        84   
2   65            1          2       1.75         -0.25        80   
3   65            2          2        0.5         -0.25       100   
4   66            1          1      -0.75         -0.50         1   

   ∆k Pentagram magnitude (D)  ∆k Pentagram axis (°)  \
0                         0.6                   84.0   
1                         0.7                  123.6   
2                         0.1                  157.4   
3                         0.4                   89.5   
4                         1.2                   83.6   

   ∆k IOL 700 magnitude (D)  ∆k IOL 700 axis (°)  ...  \
0                      0.52                   41  ...   
1                 